#Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134). Guarde o índice em disco em formato csv.

In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
import seaborn as sns
import heapq
import time
from nltk.tokenize import RegexpTokenizer
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
!pip install memory_profiler
%load_ext memory_profiler
nltk.download('stopwords')
nltk.download('rslp')

results = pd.read_csv('https://raw.githubusercontent.com/BielMaracaja/RI/master/ri_lab_02/results.csv')
documents = results['text']
stopwords = nltk.corpus.stopwords.words('portuguese') 

tokenizador = RegexpTokenizer(r'\b[A-zÀ-ú\d\-\']+')
indices = {}
n = 0

for documento in documents:
    n += 1
    tokens = tokenizador.tokenize(documento.lower())
    for token in set(tokens):
        if (token not in stopwords and len(token) >= 2):
            contador = (n, tokens.count(token))
            if (token not in indices.keys()):
                  indices[token] = []
                  indices[token].append(contador)
            else:
                  indices[token].append(contador)

                
#Indice em formato CSV sendo criado
inverted_index_df = pd.DataFrame.from_dict(indices, orient="index")
inverted_index_df.to_csv('data.csv')
indices = indices

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


#Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez (Fig. 5.16 e 5.18).

1- Defina 5 consultas de um termo somente.

2- Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).

3- Dê evidências de que sua implementação está correta.

4- Compare os tempos médios de execução e uso de memória de cada algoritmo.

In [23]:
#Busca documento_no_time sendo implementada
def documento_no_time(consulta, indices, k):
    
    listas_invertidas = []
    r = []
    
    for palavra in consulta.split(" "):
        if palavra in indices.keys():
            listas_invertidas.append(indices[palavra])
    
    for documento in range(1, len(documents)+1):
        sd = 0
        for lista_invertida in listas_invertidas:
            for i in lista_invertida:
                if (i[0] == documento): 
                    sd += i[1]
                    break
        
        if (sd != 0):
          heapq.heappush(r, (sd, documento))
    
    return heapq.nlargest(k, r)
 
documento_no_time("educação", indices, 10)

[(22, 221),
 (11, 222),
 (7, 130),
 (6, 239),
 (5, 160),
 (5, 37),
 (4, 215),
 (4, 110),
 (3, 233),
 (3, 205)]

In [24]:
#Busca termo_no_time sendo implementada
def termo_no_time(consulta, indices, k):
    
    a = {}
    listas_invertidas = []
    r = []
    
    for palavra in consulta.split(" "):
        if palavra in indices.keys():
            listas_invertidas.append(indices[palavra])
    
    for lista_invertida in listas_invertidas:
        for p in lista_invertida:
            d = p[0]
            freq = p[1]
            
            if (d in a.keys()):
                a[d] += freq
            else:
                a[d] = freq
    
    for (d, ad) in a.items():
        sd = ad
        heapq.heappush(r, (sd, d))
    
    return heapq.nlargest(k, r)
   
termo_no_time("educação", indices, 10)

[(22, 221),
 (11, 222),
 (7, 130),
 (6, 239),
 (5, 160),
 (5, 37),
 (4, 215),
 (4, 110),
 (3, 233),
 (3, 205)]

In [0]:
#Definição das buscas
buscas = ["bolsonaro", "política", "negócios", "empresa", "educação"]

In [0]:
resultados_documento = []

resultados_termo = []

resultados_documento_tempo = []

resultados_termo_tempo = []

k = 10

for busca in buscas:
  inicio_documento = time.time()
  resultados_documento.append(documento_no_time(busca, indices, k))

  fim_documento = time.time()
  resultados_documento_tempo.append(fim_documento - inicio_documento)
  
  inicio_termo = time.time()
  resultados_termo.append(termo_no_time(busca, indices, k))
  
  fim_termo = time.time()
  resultados_termo_tempo.append(fim_termo - inicio_termo)

In [27]:
#Tabela com dados sendo impressa
tabela_consultas = pd.DataFrame()

tabela_consultas['consulta'] = buscas

tabela_consultas['documento_no_time'] = resultados_documento

tabela_consultas['termo_no_time'] = resultados_termo

tabela_consultas['verdadeiro_ou_falso'] = tabela_consultas.documento_no_time == tabela_consultas.termo_no_time

tabela_consultas.index += 1

tabela_consultas

,consulta,documento_no_time,termo_no_time,verdadeiro_ou_falso
1,bolsonaro,"[(46, 151), (37, 166), (35, 207), (26, 19), (1...","[(46, 151), (37, 166), (35, 207), (26, 19), (1...",True
2,política,"[(9, 69), (7, 235), (7, 166), (6, 204), (5, 17...","[(9, 69), (7, 235), (7, 166), (6, 204), (5, 17...",True
3,negócios,"[(3, 26), (2, 130), (2, 126), (2, 8), (1, 197)...","[(3, 26), (2, 130), (2, 126), (2, 8), (1, 197)...",True
4,empresa,"[(10, 142), (10, 40), (9, 143), (8, 22), (7, 1...","[(10, 142), (10, 40), (9, 143), (8, 22), (7, 1...",True
5,educação,"[(22, 221), (11, 222), (7, 130), (6, 239), (5,...","[(22, 221), (11, 222), (7, 130), (6, 239), (5,...",True


In [28]:
#Tempo médio de execução sendo comparados
time_table = pd.DataFrame()

time_table['um_documento_por_vez_tempo_medio'] = resultados_documento_tempo

time_table['um_termo_por_vez_tempo_medio'] = resultados_termo_tempo

time_table.index += 1

time_table

,um_documento_por_vez_tempo_medio,um_termo_por_vez_tempo_medio
1,0.001116,0.000082
2,0.001301,0.000105
3,0.000316,0.000024
4,0.003657,0.000190
5,0.000717,0.000057


In [29]:
#Uso de memória sendo comparado
print('Cada documento, usando a memória por vez:')
%memit documento_no_time(buscas[0], indices, k)

print(' ')

print('Cada Termo, usando a memória por vez:')
%memit termo_no_time(buscas[0], indices, k)

Cada documento, usando a memória por vez:
peak memory: 308.68 MiB, increment: 0.00 MiB
 
Cada Termo, usando a memória por vez:
peak memory: 308.68 MiB, increment: 0.00 MiB


#Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21).

1- Defina 5 consultas conjuntivas (AND).

2- Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).

3- Dê evidências de que sua implementação está correta.

In [0]:
#Implementação da consulta conjuntiva (AND)
def and_consult(consulta, indices, k):
  
  indices_consulta = []
  ranking = [] 
  
  for palavra in consulta.split(" "):
    
    if palavra in indices.keys():
        indices_consulta.append(indices[palavra])
  
  todos_os_indices = [item for sublista in indices_consulta for item in sublista]
  todos_os_indices.sort() 
  
  for i in range(len(todos_os_indices)):
    score_documento = 0
    d = todos_os_indices.pop()
    cont = 1
    
    for indice in todos_os_indices:
      
      if indice[0] == d[0]:
        score_documento += indice[1]
        cont += 1
    
    if score_documento != 0 and cont == len(indices_consulta):
      score_documento += d[1]
      heapq.heappush(ranking, (score_documento, d[0]))
        
  return heapq.nlargest(k, ranking)

and_consults = ["bolsonaro", "política", "negócios", "empresa", "educação"]

In [31]:
#Imprimindo a tabela com os devidos dados dos documentos
k = 10
resultados_and = []

for consult in and_consults:
  score_and = and_consult(consult, indices, k)
  resultados_and.append(score_and)


and_table = pd.DataFrame()

and_table['busca'] = buscas

and_table['AND'] = resultados_and

and_table.index += 1

and_table

,busca,AND
1,bolsonaro,[]
2,política,[]
3,negócios,[]
4,empresa,[]
5,educação,[]


**3.3**
Podemos perceber a lista invertida de cada termo e percebemos que todos os termos aparecem apenas no primeiro documento. De tal forma que o algoritmo faz a soma dos scores de cada termo, atribuindo o resultado do documento ao score.

In [32]:
consulta = "governo busca dedicar educação"

termos_consulta = consulta.split()

indices_termos = [indices[termo] for termo in termos_consulta]
  
score_and = consulta_and(consulta, indices, 10)


termos_consulta.append('')
indices_termos.append('')
a = ['', '', '', '', consulta]
b = ['', '', '', '', score_and]

tabela_q = pd.DataFrame()

tabela_q['termos'] = termos_consulta

tabela_q['lista_invertida'] = indices_termos

tabela_q['consulta'] = a

tabela_q['resultado'] = b

tabela_q.index += 1

tabela_q

,termos,lista_invertida,consulta,resultado
1,governo,"[(3, 2), (8, 5), (17, 1), (18, 2), (19, 8), (2...",,
2,busca,"[(6, 3), (29, 1), (36, 1), (49, 1), (57, 2), (...",,
3,dedicar,"[(9, 1), (36, 1), (123, 1), (177, 1), (187, 1)]",,
4,educação,"[(3, 1), (18, 1), (32, 1), (37, 5), (87, 1), (...",,
5,,,governo busca dedicar educação,[]
